Motivation: individual gross outliers from general station distribution are a common error in obs data by random recording, reporting, formatting, or instrumentation errors

Process:
1. uses individual observation deviations derived from monthly mean climatology calculated for each hour of the day
2. climatologies calculated using winsorised data to remove initial effect of outliers
    - Winsorising: all values beyond threhsold value from mean are set to that threshold value
    - 5 and 95% for hadisd
    - number of data values in population remains the same, not trimmed
3. raw unwinsorised observations are anomalised using these climatologies
4. standardized by IQR for that month and hour
    - IQR cannot be less than 1.5degC
5. values are low-pass filtered to remove any climate change signal causing overzealous removal at ends of time series
6. gaussian is fitted to the histogram of anomalies for each month
7. threshold value, rounded outwards where crosses y=0.1 line
8. distribution beyond threhsold value is scanned for gap, equal to bin width or more
9. all values beyond gap are flagged
10. obs that fall between critical threshold value and gap or critical threshold and end of distribution are tentatively flagged
    - these may be later reinstated on comparison with good data from neighboring stations

Notes:
- when applied to SLP, frequently flags storm signals, which may be of high interest, so this test is not applied to pressure data
- hadisd only applies to temp and dewpoint temp

In [2]:
import pandas as pd
import numpy as np
import xarray as xr

from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [36]:
ds = xr.open_dataset('/Users/victoriaford/Desktop/eaglerock/Historical Data Platform/Train_Files/ASOSAWOS_72051724165.nc')

df = ds.to_dataframe()
df = df.reset_index()
df['month'] = pd.to_datetime(df['time']).dt.month # sets month to new variable
df['year'] = pd.to_datetime(df['time']).dt.year # sets year to new variable

/Users/victoriaford/anaconda3/envs/histobs/lib/python3.11/site-packages/pandas/core/arrays/timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
/Users/victoriaford/anaconda3/envs/histobs/lib/python3.11/site-packages/pandas/core/arrays/timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


In [136]:
def clim_mon_mean_hourly(df, var, month, hour):
    '''Calculate the monthly mean climatology for each of the day'''
    
    df_m_h = df.loc[(df.time.dt.month == month) & (df.time.dt.hour == hour)]
    clim_value = df_m_h[var].mean(numeric_only = True)
    
    # special handling if value is nan? 
    
    return clim_value

def iqr_range_monhour(df, var, month, hour):
    '''Calculates the monthly interquartile range per hour'''
    
    q1 = df.loc[(df.time.dt.month == month) & (df.time.dt.hour == hour)].quantile(0.25, numeric_only=True)
    q3 = df.loc[(df.time.dt.month == month) & (df.time.dt.hour == hour)].quantile(0.75, numeric_only=True)
    
    iqr_df = q3 - q1
    iqr_df_val = iqr_df[var]
    
    # iqr cannot be less than 1.5°C in order to preserve low variance stations
    if iqr_df_val < 1.5:
        iqr_df_val = 1.5
    else:
        iqr_df_val = iqr_df_val
            
    return iqr_df_val


def clim_standardized_anom(df, vars_to_anom):
    '''
    First anomalizes data by monthly climatology for each hour, then
    standardizes by the monthly climatological anomaly IQR for each hour
    '''
    
    df2 = df.copy()
    
    for var in vars_to_anom:
        for m in range(1,13,1):
            for h in range(0,24,1):
                # each hour in each month
                anom_value = clim_mon_mean_hourly(df, var, month=m, hour=h)
                iqr_value = iqr_range_monhour(df, var, month=m, hour=h)
                
                # locate obs within specific month/hour
                df_m_h = df.loc[(df.time.dt.month == m) & (df.time.dt.hour == h)]
                
                # calculate the monthly climatological anomaly by hour and standardize by iqr
                df2.loc[(df.time.dt.month == m) & 
                        (df.time.dt.hour == h), 
                        var] = (df_m_h[var] - anom_value) / iqr_value
                
    return df2

def winsorize_temps(df, vars_to_anom, winz_limits):
    '''
    Replaces potential spurious outliers by limiting the extreme values
    using the winz_limits set (default is 5% and 95% percentiles)
    '''
    
    df2 = df.copy()
    
    for var in vars_to_anom:
        for m in range(1,13,1):
            for h in range(0,24,1):
                
                df_m_h = df.loc[(df.time.dt.month == m) & (df.time.dt.hour == h)]
                
                # winsorize only vars in vars_to_anom
                df_w = winsorize(df_m_h[var], limits=winz_limits, nan_policy='omit')
                
                df2.loc[(df.time.dt.month == m) & (df.time.dt.hour == h),
                       var] = df_w
                
    return df2

# get average anomaly per year
def median_yr_anom(df, var):
    '''Get median anomaly per year'''
    
    monthly_anoms = []
    
    # identify years in data
    years = df.time.dt.year.unique()
    
    for yr in years:
        df_yr = df.loc[df.time.dt.year == yr]

        ann_anom = df_yr[var].median()
        monthly_anoms.append(ann_anom)
        
    return monthly_anoms

def low_pass_filter_weights(median_anoms, month_low, month_high, filter_low, filter_high):
    '''Calculates weights for low pass filter'''
    
    filter_wgts = [1, 2, 3, 2, 1]
    
    if np.sum(filter_wgts[filter_low:filter_high] * 
              np.ceil(median_anoms[month_low:month_high] - 
                      np.floor(median_anoms[month_low:month_high]))) == 0:
        weight = 0
    
    else:
        weight = (
            np.sum(filter_wgts[filter_low:filter_high] * np.ceil(median_anoms[month_low:month_high])) / 
            np.sum(filter_wgts[filter_low:filter_high] * np.ceil(median_anoms[month_low:month_high] - 
                                                                 np.floor(median_anoms[month_low:month_high])))
        )
        
    return weight

def low_pass_filter(df, vars_to_anom):
    '''
    Low pass filtering on observations to remove any climate change signal 
    causing overzealous removal at ends of time series
    '''
    # identify years in data
    years = df.time.dt.year.unique()
    
    for var in vars_to_anom:
        
        median_anoms = median_yr_anom(df, var)
    
        for yr in range(len(years)):
            if yr == 0:
                month_low, month_high = 0, 3
                filter_low, filter_high = 2, 5
                
            elif yr == 1:
                month_low, month_high = 0, 4
                filter_low, filter_high = 1, 5
                
            elif yr == len(years)-2:
                month_low, month_high = -4, -1
                filter_low, filter_high = 0, 3

            elif yr == len(years)-1:
                month_low, month_high = -3, -1
                filter_low, filter_high = 0, 2

            else:
                month_low, month_high = yr-2, yr+3
                filter_low, filter_high = 0, 5
                            
            print(month_low, month_high, filter_low, filter_high)
            if np.sum(np.abs(median_anoms[month_low:month_high])) != 0:
                weights = low_pass_filter_weights(median_anoms, month_low, month_high, filter_low, filter_high)
                      
            # want to return specific year of data at a specific variable, the variable minus weight value
            df.loc[(df.time.dt.year == years[yr]), var] = df.loc[df.time.dt.year == years[yr]][var] - weights
            
    return df

In [155]:
vars_to_check = ['tas', 'tdps', 'tdps_derived']
vars_to_anom = [v for v in vars_to_check if v in df.columns]
df2 = winsorize_temps(df, vars_to_anom, winz_limits=[0.05,0.05])
df2 = clim_standardized_anom(df2, vars_to_anom)
df3 = low_pass_filter(df2, vars_to_anom)

0 3 2 5
weights 1.0
0 4 1 5
weights 1.0
0 5 0 5
weights 1.0
1 6 0 5
weights 1.0
2 7 0 5
weights 0.8888888888888888
3 8 0 5
weights 0.6666666666666666
4 9 0 5
weights 0.4444444444444444
5 10 0 5
weights 0.3333333333333333
-4 -1 0 3
weights 0.5
-3 -1 0 2
weights 0.6666666666666666
0 3 2 5
weights 1.0
0 4 1 5
weights 1.0
0 5 0 5
weights 1.0
1 6 0 5
weights 0.8888888888888888
2 7 0 5
weights 0.6666666666666666
3 8 0 5
weights 0.3333333333333333
4 9 0 5
weights 0.1111111111111111
5 10 0 5
weights 0.0
-4 -1 0 3
weights 0.0
-3 -1 0 2
weights 0.0


,station,time,ps,tas,tdps,pr,sfcWind,sfcWind_dir,elevation,qaqc_process,...,pr_qc,pr_duration,pr_depth_qc,sfcWind_qc,sfcWind_method,sfcWind_dir_qc,lat,lon,month,year
0,ASOSAWOS_72051724165,2013-06-27 12:55:00,78690.0,-0.026072,-0.269416,NaN,5.7,350.0,2220.0,V020,...,,NaT,NaN,5,N,5,41.824,-110.556,6,2013
1,ASOSAWOS_72051724165,2013-06-27 13:15:00,NaN,-0.334985,-0.450441,NaN,5.7,350.0,2220.0,V020,...,,NaT,NaN,1,N,1,41.824,-110.557,6,2013
2,ASOSAWOS_72051724165,2013-06-27 13:35:00,NaN,0.015015,-0.400441,NaN,4.6,350.0,2220.0,V020,...,,NaT,NaN,1,N,1,41.824,-110.557,6,2013
3,ASOSAWOS_72051724165,2013-06-27 13:55:00,78750.0,0.290015,-0.275441,NaN,4.6,350.0,2220.0,V020,...,,NaT,NaN,5,N,5,41.824,-110.556,6,2013
4,ASOSAWOS_72051724165,2013-06-27 14:15:00,NaN,-0.241029,-0.443765,NaN,4.6,360.0,2220.0,V020,...,,NaT,NaN,1,N,1,41.824,-110.557,6,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178389,ASOSAWOS_72051724165,2022-08-11 20:15:00,78640.0,-0.158290,0.454748,NaN,7.2,260.0,2220.0,V020,...,,NaT,NaN,5,N,5,41.824,-110.556,8,2022
178390,ASOSAWOS_72051724165,2022-08-11 20:35:00,78580.0,0.041710,0.829748,NaN,9.8,240.0,2220.0,V020,...,,NaT,NaN,5,N,5,41.824,-110.556,8,2022
178391,ASOSAWOS_72051724165,2022-08-11 20:55:00,78580.0,0.041710,0.829748,NaN,5.7,290.0,2220.0,V020,...,,NaT,NaN,5,V,5,41.824,-110.556,8,2022
178392,ASOSAWOS_72051724165,2022-08-11 21:15:00,78530.0,0.070496,0.834145,NaN,10.3,250.0,2220.0,V020,...,,NaT,NaN,5,N,5,41.824,-110.556,8,2022


In [ ]:
def qaqc_climatological_outlier(df, winsorize=True, winz_limits=[0.05,0.05], plot=True, verbose=True):
    '''
    Flags individual gross outliers from climatological distribution.
    Only applied to air temperature and dew point temperature
    
    Input:
    ------
        df [pd.DataFrame]: station dataset converted to dataframe through QAQC pipeline
        plots [bool]: if True, produces plots of any flagged data and saved to AWS
            
    Returns:
    --------
        qaqc success:
            df [pd.DataFrame]: QAQC dataframe with flagged values (see below for flag meaning)
        qaqc failure:
            None
            
    Flag meaning:
    -------------
        25,qaqc_climatological_outlier,Value flagged as a climatological outlier
    '''
    
    #### ONLY IN NOTEBOOK DEVELOPMENT, REMOVED IN CODE FOR PIPELINE #####
    df = df.reset_index()
    df['month'] = pd.to_datetime(df['time']).dt.month # sets month to new variable
    df['year'] = pd.to_datetime(df['time']).dt.year # sets year to new variable
    
    vars_to_check = ['tas', 'tdps', 'tdps_derived']
    vars_to_anom = [v for v in vars_to_check if v in df.columns]
    
    # TO DO: filter to only use non-flagged data

    
    # winsorize data by percentiles
    if winsorize == True:
        df = winsorize_temps(df, vars_to_anom, winz_limits)
    else:
        df = df
        
    # standardize data by monthly climatological anomalies by hour
    df = clim_standardized_anom(df, vars_to_anom)

    # apply low pass filter
    df = low_pass_filter(df, vars_to_anom)
        
    # gaussian is fitted to the histogram of anomalies for each month
        # threshold value, rounded outwards where crosses y=0.1 line
        # distribution beyond threhsold value is scanned for gap, equal to bin width or more
        # all values beyond gap are flagged
            # HadISD: obs that fall between critical threshold value and gap or 
            # critical threshold and end of distribution are tentatively flagged
            # these may be later reinstated on comparison with good data from neighboring stations
            # into v2 of data product